In [15]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from numpy.random import choice

# read data
data = pd.read_feather("dataFrame.feather")
# encode categorical variable
le = LabelEncoder()
data["category"] = le.fit_transform(data["category"])
# train-test split
TRAIN_RATIO = 0.8
train_ind = choice(data.shape[0], size = int(data.shape[0]*TRAIN_RATIO), replace = False)
train_dt = data.iloc[train_ind,:]
test_dt = data.drop(data.index[train_ind])

In [ ]:
OUTPUT_DIR = "Desktop/output"
DATA_COLUMN = 'text'
LABEL_COLUMN = 'category'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1, 2, 3, 4]device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

In [34]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
# torch.cuda.get_device_name(0)

In [30]:
train_sentence = train_dt.loc[:,DATA_COLUMN]
test_sentence = test_dt.loc[:,DATA_COLUMN]
train_labels = train_dt.loc[:,LABEL_COLUMN]
test_labels = test_dt.loc[:,LABEL_COLUMN]

In [39]:
# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
train_sentence = ["[CLS] " + sentence + " [SEP]" for sentence in train_sentence]
test_sentence = ["[CLS] " + sentence + " [SEP]" for sentence in test_sentence]


In [42]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_train_texts = [tokenizer.tokenize(sent) for sent in train_sentence]
tokenized_test_texts = [tokenizer.tokenize(sent) for sent in test_sentence]
print ("Tokenize the first sentence:")
print (tokenized_train_texts[0])

Tokenize the first sentence:
['[CLS]', '[CLS]', 'wine', 'comedy', 'wins', 'critics', "'", 'award', 'qui', '##rky', 'comedy', 'sideways', 'was', 'named', 'the', 'best', 'film', 'of', 'the', 'year', 'by', 'the', 'los', 'angeles', 'film', 'critics', 'association', '.', 'the', 'us', 'movie', 'also', 'picked', 'up', 'four', 'other', 'accolades', 'including', 'best', 'director', 'for', 'alexander', 'payne', 'and', 'supporting', 'actor', 'for', 'thomas', 'had', '##en', 'church', '.', 'british', 'actress', 'im', '##eld', '##a', 'st', '##au', '##nton', 'has', 'again', 'been', 'recognised', 'for', 'her', 'role', 'in', 'vera', 'drake', ',', 'winning', 'best', 'actress', ',', 'while', 'liam', 'nee', '##son', 'won', 'best', 'actor', 'for', 'kin', '##sey', '.', 'the', 'awards', 'will', 'be', 'handed', 'out', 'on', '13', 'january', 'at', 'a', 'ceremony', 'in', 'las', 'vegas', '.', 'sideways', 'tells', 'the', 'story', 'of', 'two', 'men', 'who', 'take', 'a', 'road', 'trip', 'through', 'california', "'"

In [48]:

# index word encoding
input_train_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_train_texts]
input_test_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_test_texts]


In [49]:
# padding
MAX_LEN = 512
input_train_ids = pad_sequences(input_train_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
input_test_ids = pad_sequences(input_test_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [51]:
# Create attention masks
train_attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_train_ids:
  seq_mask = [float(i>0) for i in seq]
  train_attention_masks.append(seq_mask)
    
    # Create attention masks
test_attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_test_ids:
  seq_mask = [float(i>0) for i in seq]
  test_attention_masks.append(seq_mask)

In [57]:
train_inputs = torch.tensor(input_train_ids)
validation_inputs = torch.tensor(input_test_ids)

train_labels = torch.tensor(train_labels.values)
validation_labels = torch.tensor(test_labels.values)

train_masks = torch.tensor(train_attention_masks)
validation_masks = torch.tensor(test_attention_masks)

In [59]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [61]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=5)

In [62]:
# !!!!!!!!!!!!!!!!!!!!!!!! ZROZUM TEN FRAGMENT
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [63]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

In [64]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [66]:
t = [] 

# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/4 [00:01<?, ?it/s]


RuntimeError: [enforce fail at CPUAllocator.cpp:64] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 402653184 bytes. Error code 12 (Cannot allocate memory)
